<a href="https://colab.research.google.com/github/thimotyb/real-world-machine-learning/blob/python3/Importing_data_with_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Dimostrazione di collegamento a database usando SQLLite

import pandas as pd
import sqlite3
conn = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airlines limit 5;", conn)
df

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [23]:
# Prelievo senza dataframe
cur = conn.cursor()
cur.execute("select * from airlines limit 5;")
results = cur.fetchall()
print(results)

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'), (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'), (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'), (3, '4', '2 Sqn No 1 Elementary Flying Training School', '\\N', None, 'WYT', None, 'United Kingdom', 'N'), (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]


In [24]:
# Inserimento
cur.execute("insert into airlines values (6048, 19846, 'Test flight', '', '', null, null, null, 'Y')")


In [25]:
cur.execute("select * from airlines where name = 'Test flight';")
results = cur.fetchall()
print(results)

[(6048, '19846', 'Test flight', '', '', None, None, None, 'Y'), (6048, '19846', 'Test flight', '', '', None, None, None, 'Y')]


In [26]:
# Uso di parametri per evitare sql injection

# Non così
#name = 'Test Flight'
#cur.execute("insert into airlines values (6049, 19847, {0}, '', '', null, null, null, 'Y')".format(name))

# Ma così
values = ('Test Flight', 'Y')
cur.execute("insert into airlines values (6049, 19847, ?, '', '', null, null, null, ?)", values)

In [30]:
cur.execute("select * from airlines where name = 'Test Flight';")
results = cur.fetchall()
print(results)

[(6049, '19847', 'Test Flight', '', '', None, None, None, 'Y'), (6049, '19847', 'Test Flight', '', '', None, None, None, 'Y')]


In [31]:
# Creare tabelle direttamente con SQL
cur.execute("create table daily_flights (id integer, departure date, arrival date, number text, route_id integer)")
cur.execute("insert into daily_flights values (1, '2016-09-28 0:00', '2016-09-28 12:00', 'T1', 1)")


In [32]:
pd.read_sql_query("select * from daily_flights;", conn)

,id,departure,arrival,number,route_id
0,1,2016-09-28 0:00,2016-09-28 12:00,T1,1


In [33]:
# Creare tabelle e manipolare dati in SQL con Pandas
from datetime import datetime
df = pd.DataFrame(
[[1, datetime(2016, 9, 29, 0, 0) ,
datetime(2016, 9, 29, 12, 0), 'T1', 1]],
columns=["id", "departure", "arrival", "number", "route_id"])

In [34]:
df.to_sql("daily_flights", conn, if_exists="replace")

In [35]:
pd.read_sql_query("select * from daily_flights;", conn)

,index,id,departure,arrival,number,route_id
0,0,1,2016-09-29 00:00:00,2016-09-29 12:00:00,T1,1


In [36]:
# Possibilità di Alter Table
df = pd.read_sql("select * from daily_flights", conn)
df["delay_minutes"] = None
df.to_sql("daily_flights", conn, if_exists="replace")

In [37]:
df

,index,id,departure,arrival,number,route_id,delay_minutes
0,0,1,2016-09-29 00:00:00,2016-09-29 12:00:00,T1,1,None
